## 📦 Installation des dépendances

In [ ]:
# Installer les packages nécessaires
!pip install -q transformers torch accelerate datasets

## ✅ Vérification GPU

In [ ]:
import torch

# Vérifier GPU
if torch.cuda.is_available():
    print(f"✅ GPU disponible: {torch.cuda.get_device_name(0)}")
    print(f"   Mémoire totale: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
else:
    print("❌ GPU non disponible - Activer le GPU dans Runtime settings!")

## 📥 Création du Dataset bien-être

500 exemples répartis en 5 classes de sentiments.

In [ ]:
import random
import json

def create_wellbeing_dataset(size=500):
    """
    Crée un dataset équilibré de sentiments bien-être
    """
    
    WELLBEING_EXAMPLES = {
        'très négatif': [
            "Je veux tout abandonner", "Je ne vois pas d'issue", "Je suis désespéré",
            "Je préfère ne pas exister", "Je ne mérite pas de vivre", "Je suis un fardeau",
            "Rien n'a de sens", "Je veux disparaître", "C'est trop pour moi",
            "Je ne peux plus continuer", "Tout est noir", "Je suis incapable",
            "Personne ne m'aime", "Je suis perdu", "Je veux en finir",
            "La vie n'a aucune valeur", "Je suis un échec", "Je ne vaux rien",
            "Je suis seul et abandonné", "Tout est inutile",
        ],
        'négatif': [
            "Je suis triste", "Rien n'a d'importance", "Je me sens vide",
            "Je suis stressé", "Je suis anxieux", "Je me sens mal",
            "J'ai peur", "Je suis découragé", "Ça ne va pas bien",
            "Je suis fatigué", "Je ne sais pas quoi faire", "Je me sens seul",
            "C'est trop difficile", "Je suis frustré", "Je n'ai pas d'énergie",
            "Je suis déprimé", "Rien ne me plaît", "Je suis inquiet",
            "Je doute de moi", "C'est déprimant", "Je suis submergé",
            "Tout est compliqué", "Je suis affecté", "J'ai des pensées négatives",
            "Ça m'angoisse",
        ],
        'neutre': [
            "Bonjour, comment ça va?", "Il fait beau dehors", "Quelle heure est-il?",
            "Ça va, et toi?", "Qu'est-ce que tu fais?", "Je ne sais pas",
            "C'est normal", "C'est juste une journée ordinaire", "Rien de spécial",
            "C'est comme d'habitude", "Je fais juste passer le temps", "C'est banal",
            "Rien de nouveau", "C'est la routine", "Pas grand-chose",
            "Juste ça", "C'est ça", "Comme d'habitude",
            "Rien qui change", "C'est tout", "Je vais bien, merci",
            "Aucun souci", "C'est ok", "Pas mal", "C'est acceptable",
        ],
        'positif': [
            "Ça va plutôt bien", "J'ai une bonne journée", "Je suis content",
            "Je me sens mieux", "C'est sympa", "J'aime bien",
            "Je suis fier de moi", "J'ai du courage", "Je peux le faire",
            "Ça va s'arranger", "J'ai de l'espoir", "Je suis motivé",
            "C'est agréable", "Je me sens bien", "J'ai de l'énergie",
            "C'est cool", "Je suis optimiste", "Ça me plaît",
            "Je suis confiant", "C'est positif", "Je me sens fort",
            "J'ai du potentiel", "Ça me fait du bien", "Je suis satisfait",
            "Ça marche bien",
        ],
        'très positif': [
            "Je suis heureux!", "C'est magnifique!", "Je suis aux anges!",
            "C'est formidable!", "Je suis ravi!", "C'est incroyable!",
            "Je suis enthousiaste!", "C'est génial!", "Je suis exubérant!",
            "C'est fantastique!", "Je suis débordant de joie!", "C'est merveilleux!",
            "Je suis dans l'euphorie!", "C'est excellent!", "Je suis passionné!",
            "C'est extraordinaire!", "Je suis conquis!", "C'est superbe!",
            "Je suis comblé!", "C'est spectaculaire!", "Je suis sur un nuage!",
            "C'est sublime!", "Je suis épanoui!", "C'est merveilleux!",
            "Je suis vivant et énergique!",
        ]
    }
    
    dataset = []
    label_to_id = {
        'très négatif': 0, 'négatif': 1, 'neutre': 2,
        'positif': 3, 'très positif': 4
    }
    
    examples_per_class = size // 5
    
    for label, examples in WELLBEING_EXAMPLES.items():
        if len(examples) >= examples_per_class:
            selected = random.sample(examples, examples_per_class)
        else:
            selected = examples * (examples_per_class // len(examples) + 1)
            selected = selected[:examples_per_class]
        
        for text in selected:
            dataset.append({
                'text': text,
                'label': label,
                'label_id': label_to_id[label]
            })
    
    random.shuffle(dataset)
    return dataset

# Créer le dataset
dataset = create_wellbeing_dataset(500)
print(f"✅ Dataset créé: {len(dataset)} exemples")

# Statistiques
from collections import Counter
label_counts = Counter(d['label'] for d in dataset)
for label, count in label_counts.items():
    print(f"   {label:20s}: {count:3d} exemples")

## 📊 Split Train/Validation (80/20)

In [ ]:
# Split train/val
split_idx = int(len(dataset) * 0.8)
train_dataset = dataset[:split_idx]
val_dataset = dataset[split_idx:]

train_texts = [d['text'] for d in train_dataset]
train_labels = [d['label_id'] for d in train_dataset]

val_texts = [d['text'] for d in val_dataset]
val_labels = [d['label_id'] for d in val_dataset]

print(f"✅ Train: {len(train_texts)} exemples")
print(f"✅ Validation: {len(val_texts)} exemples")

## 🤖 Création du Dataset PyTorch

In [ ]:
from torch.utils.data import Dataset

class WellbeingDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        
        encoding = self.tokenizer(
            text,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        
        return {
            'input_ids': encoding['input_ids'].squeeze(),
            'attention_mask': encoding['attention_mask'].squeeze(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

print("✅ Classe WellbeingDataset créée")

## 🔧 Chargement du modèle BERT

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_name = 'bert-base-multilingual-uncased'

print(f"📥 Chargement de {model_name}...")

# Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
print("✅ Tokenizer chargé")

# Modèle (5 labels)
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=5
)
print("✅ Modèle BERT chargé (110M paramètres)")

# Placer sur GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
print(f"✅ Modèle sur: {device}")

## 🎯 Préparation des datasets PyTorch

In [ ]:
# Créer les datasets
train_pytorch_dataset = WellbeingDataset(train_texts, train_labels, tokenizer)
val_pytorch_dataset = WellbeingDataset(val_texts, val_labels, tokenizer)

print(f"✅ Train dataset: {len(train_pytorch_dataset)} exemples")
print(f"✅ Val dataset: {len(val_pytorch_dataset)} exemples")

## 🔥 Configuration de l'entraînement

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir='./bert_finetuned',
    num_train_epochs=3,              # 3 epochs
    per_device_train_batch_size=16,  # Batch 16 sur GPU
    per_device_eval_batch_size=16,
    learning_rate=2e-5,              # Learning rate standard
    weight_decay=0.01,
    eval_strategy='epoch',
    save_strategy='epoch',
    load_best_model_at_end=True,
    metric_for_best_model='eval_loss',
    logging_steps=10,
    seed=42,
    fp16=True,                       # Mixed precision sur GPU
)

print("✅ Configuration prête")

## 🚀 LANCER L'ENTRAÎNEMENT

**Durée estimée: 2-3 minutes avec GPU T4**

In [ ]:
# Créer le Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_pytorch_dataset,
    eval_dataset=val_pytorch_dataset,
)

print("🔥 LANCEMENT DU FINE-TUNING...\n")

# ENTRAÎNER
trainer.train()

print("\n✅ ENTRAÎNEMENT TERMINÉ!")

## 💾 Sauvegarder le modèle

In [ ]:
# Sauvegarder
output_dir = './bert_finetuned_final'
trainer.save_model(output_dir)
tokenizer.save_pretrained(output_dir)

print(f"✅ Modèle sauvegardé dans: {output_dir}")

## 🧪 Tester le modèle fine-tuné

In [ ]:
# Fonction de prédiction
SENTIMENT_MAP = {
    0: 'très négatif',
    1: 'négatif',
    2: 'neutre',
    3: 'positif',
    4: 'très positif'
}

def predict(text):
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=128)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    
    with torch.no_grad():
        outputs = model(**inputs)
    
    logits = outputs.logits
    probabilities = torch.softmax(logits, dim=-1)
    predicted_id = torch.argmax(probabilities).item()
    confidence = probabilities[0, predicted_id].item()
    
    return {
        'sentiment': SENTIMENT_MAP[predicted_id],
        'confidence': confidence
    }

# Tests
test_phrases = [
    "Je suis heureux!",
    "Je me sens déprimé",
    "Comment ça va?",
    "Je ne veux plus continuer",
    "C'est magnifique!",
    "Je suis très stressé",
    "Tout va bien",
    "Je suis désespéré",
]

print("\n📊 TESTS DU MODÈLE FINE-TUNÉ")
print("=" * 70)

for phrase in test_phrases:
    result = predict(phrase)
    emoji_map = {
        'très négatif': '🔴',
        'négatif': '🟠',
        'neutre': '🟡',
        'positif': '🟢',
        'très positif': '🟢🟢'
    }
    emoji = emoji_map[result['sentiment']]
    print(f"{emoji} '{phrase}'")
    print(f"   → {result['sentiment']:15s} ({result['confidence']:.1%})")
    print()

## 📥 Télécharger le modèle

Compresse le modèle pour le télécharger sur votre ordinateur.

In [ ]:
# Compresser le modèle
!zip -r bert_finetuned_final.zip bert_finetuned_final/

print("✅ Modèle compressé: bert_finetuned_final.zip")
print("\n📥 Télécharger avec:")
print("   - Clic droit sur le fichier dans l'explorateur Colab")
print("   - Ou utiliser Google Drive pour le sauvegarder")

## 💾 Sauvegarder sur Google Drive (Optionnel)

In [ ]:
# Monter Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Copier le modèle
!cp -r bert_finetuned_final /content/drive/MyDrive/

print("✅ Modèle sauvegardé sur Google Drive: MyDrive/bert_finetuned_final/")

## 🎉 TERMINÉ!

**Prochaines étapes:**
1. Télécharger le modèle (`bert_finetuned_final.zip`)
2. Extraire dans `models/approach3/bert_finetuned/`
3. Tester avec `python compare_approaches.py`

**Résumé:**
- ✅ BERT fine-tuné sur 500 exemples bien-être
- ✅ 5 classes de sentiments
- ✅ Entraînement GPU ~2-3 minutes
- ✅ Modèle prêt pour production